Installs

In [ ]:
!pip install umap-learn
!pip install git+https://github.com/bp-kelley/descriptastorus
!pip install rdkit
!pip install torchinfo
!pip install torchmetrics

 Import libraries

In [ ]:
# Standard python libraries
import collections
import random
import itertools
import io

# Scientific python
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import scipy as sp
import scipy.spatial.distance as sp_dist
import numpy as np
import pandas as pd
import altair as alt

# Chemoinformatics
import rdkit
import rdkit.Chem.Descriptors
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem.Draw.MolDrawing import MolDrawing
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Descriptors
from rdkit.Chem import rdFingerprintGenerator
import descriptastorus
# Great progress bar
import tqdm.auto as tqdm
# ML: UMAP, PCA and clustering
import umap
import sklearn.decomposition
import sklearn.cluster
import sklearn.model_selection
import sklearn.preprocessing

# Our DL stack
import torch
import torch.nn as nn
import torch.utils.data as torch_data
import torchinfo
import torchmetrics

# Jupyter/IPython libraries
import IPython.display as ipy_display
from IPython.display import Image
import base64
import PIL

for mod in [np, sp, rdkit, torch]:
    print(f'{mod.__name__:20s}:{mod.__version__}')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\nUsing device: {device}")

In [ ]:
# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Sample dataset, solubility (logP) 💧

Datatset source: Delaney's solubility dataset from ESOL:  Estimating Aqueous Solubility Directly from Molecular Structure
(https://pubs.acs.org/doi/10.1021/ci034243x)


In [ ]:
!wget -nc https://raw.githubusercontent.com/chemcognition-lab/CHE1148/refs/heads/main/data/delaney/delaney-solubility.csv

In [ ]:
df = pd.read_csv('delaney-solubility.csv')
target_col = 'measured log solubility in mols per litre'
print(df.shape)
df.head()

Generate our input features

In [ ]:
df['mol'] = df['smiles'].apply(Chem.MolFromSmiles)
generator = descriptastorus.descriptors.rdNormalizedDescriptors.RDKit2DNormalized()
x_all = np.stack([generator.calculateMol(m, None) for m in tqdm.tqdm(df['mol'].tolist())])
y_all = df[target_col].values.reshape(-1, 1)
print(f"Features: {x_all.shape}, Targets: {y_all.shape}")

## Create splits

We generate three columns in the DataFrame indicating 'train' (80%) or 'test' (20%).

Later, the 'train' portion will be further split into Train (65%) and Val (15%).


### Random first

In [ ]:
n = len(df)
indices = list(range(n))
test_size = 0.20
df['split_random_new'] = 'train'
test_idxs = np.random.choice(n, int(n * test_size), replace=False)
df.iloc[test_idxs, df.columns.get_loc('split_random_new')] = 'test'
df['split_random_new'] = 'train'
test_idxs = np.random.choice(n, int(n * test_size), replace=False)
df.iloc[test_idxs, df.columns.get_loc('split_random_new')] = 'test'

2. Stratified Split (High LogP Threshold)

Binarize based on top 20% solubility (High vs Low/Medium)

In [ ]:
threshold = df[target_col].quantile(0.8)
df['strat_bin_new'] = (df[target_col] > threshold).astype(int)
splitter = sklearn.model_selection.StratifiedShuffleSplit(n_splits=1,
                                                          test_size=test_size,
                                                          random_state=42)
train_idx, test_idx = next(splitter.split(indices, df['strat_bin']))
df['split_stratified_new'] = 'train'
df.iloc[test_idx, df.columns.get_loc('split_stratified_new')] = 'test'

# 3. Adversarial Split (Distance/Cluster based)

We cluster the data. We explicitly assign specific clusters to 'Test'
to force the model to predict on chemically distinct groups (OOD).

In [ ]:
kmeans = sklearn.cluster.KMeans(n_clusters=10, random_state=42, n_init=10)
clusters = kmeans.fit_predict(x_all)
# Pick clusters to define the test set until we reach ~20%
cluster_counts = pd.Series(clusters).value_counts()
test_clusters = []
current_count = 0
target_count = int(n * test_size)

for clus_id, count in cluster_counts.items():
    if current_count + count <= target_count + 50: # Allow small buffer
        test_clusters.append(clus_id)
        current_count += count

df['split_adversarial_new'] = 'train'
test_mask = np.isin(clusters, test_clusters)
df.loc[test_mask, 'split_adversarial_new'] = 'test'

Visualization of the Adversarial Split


In [ ]:
pca = sklearn.decomposition.PCA(n_components=2)
x_pca = pca.fit_transform(x_all)
plt.figure(figsize=(8,6))
sns.scatterplot(x=x_pca[:,0], y=x_pca[:,1], hue=df['split_adversarial'], style=df['split_adversarial'])
plt.title("Adversarial Split (PCA View)")
plt.show()

## Data Loaders Setup

In [ ]:
def get_dataloaders(split_name, x_data, y_data, df, batch_size=64):
    """
    1. Selects 80% Train / 20% Test based on `split_name` column.
    2. Splits the 80% Train into -> 65% Train / 15% Val.
    """

    # Indices for the outer split
    train_val_indices = df[df[split_name] == 'train'].index.to_numpy()
    test_indices = df[df[split_name] == 'test'].index.to_numpy()

    # Create the internal split (Train vs Val)
    # We want Val to be 15% of TOTAL, and Train to be 65% of TOTAL.
    # Total Train+Val is 80%.
    # Fraction for val = 15 / 80 = 0.1875
    val_rel_size = 15 / 80

    train_idx, val_idx = sklearn.model_selection.train_test_split(
        train_val_indices, test_size=val_rel_size, random_state=42
    )

    print(f"Split: {split_name}")
    print(f"Train: {len(train_idx)} (65%) | Val: {len(val_idx)} (15%) | Test: {len(test_indices)} (20%)")

    # Convert to Tensors
    x_tensor = torch.tensor(x_data, dtype=torch.float32)
    y_tensor = torch.tensor(y_data, dtype=torch.float32)

    train_ds = torch_data.TensorDataset(x_tensor[train_idx], y_tensor[train_idx])
    val_ds = torch_data.TensorDataset(x_tensor[val_idx], y_tensor[val_idx])
    test_ds = torch_data.TensorDataset(x_tensor[test_indices], y_tensor[test_indices])

    return (
        torch_data.DataLoader(train_ds, batch_size=batch_size, shuffle=True),
        torch_data.DataLoader(val_ds, batch_size=batch_size, shuffle=False),
        torch_data.DataLoader(test_ds, batch_size=batch_size, shuffle=False)
    )

In [ ]:
SPLIT = 'split_random'
train_loader, val_loader, test_loader = get_dataloaders(SPLIT, x_all, y_all, df)

# Setting up a model class

In [ ]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_dim=1, dropout=0.0):
        super(MLP, self).__init__()

        self.layers = nn.ModuleList()
        # Input
        self.layers.append(nn.Linear(input_size, hidden_size))
        self.layers.append(nn.ReLU())
        self.layers.append(nn.LayerNorm(hidden_size))

        # Hidden
        for _ in range(num_layers - 1):
            self.layers.append(nn.Linear(hidden_size, hidden_size))
            self.layers.append(nn.ReLU())
            self.layers.append(nn.Dropout(p=dropout))
            self.layers.append(nn.LayerNorm(hidden_size))

        # Output
        self.predict = nn.Linear(hidden_size, output_dim)

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return self.predict(x)

We create the model based on some hyper-parameters

In [ ]:
input_dim = x_all.shape[1]
model = MLP(input_dim, hidden_size=128, num_layers=3)
model = model.to(device)
torchinfo.summary(model, input_size=(64, input_dim))

## Optimization & Metrics


In [ ]:
class EarlyStopping:
    def __init__(self, patience=10, verbose=False):
        self.patience = patience
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
        self.verbose = verbose

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss:
            self.counter += 1
            if self.verbose and self.counter % 5 == 0:
                print(f"EarlyStopping counter: {self.counter}/{self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0


Our metrics

In [ ]:

metrics_template = torchmetrics.MetricCollection({
    'r2': torchmetrics.R2Score(),
    'mae': torchmetrics.MeanAbsoluteError(),
    'spear': torchmetrics.SpearmanCorrCoef(),
})

train_metrics = metrics_template.clone().to(device)
val_metrics = metrics_template.clone().to(device)
test_metrics = metrics_template.clone().to(device)

In [ ]:
lr = 0.001
loss_fn = nn.MSELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-2)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
early_stopping = EarlyStopping(patience=15, verbose=True)
clip_value = 1.0

# Evaluation & Training loop

In [ ]:
def evaluate(model, loader, metrics_obj):
    model.eval()
    metrics_obj.reset()
    total_loss = 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            preds = model(x)
            loss = loss_fn(preds, y)
            total_loss += loss.item()
            metrics_obj.update(preds, y)

    results = metrics_obj.compute()
    results = {k: v.item() for k,v in results.items()}
    results['loss'] = total_loss / len(loader)
    return results

In [ ]:
epochs = 100
history = []

pbar = tqdm.tqdm(range(1, epochs+1))

for epoch in pbar:
    # --- TRAIN ---
    model.train()
    train_metrics.reset()
    train_loss_acc = 0

    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        preds = model(x)
        loss = loss_fn(preds, y)
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip_value)
        optimizer.step()

        train_loss_acc += loss.item()
        train_metrics.update(preds, y)

    # --- EVALUATE ---
    train_res = train_metrics.compute()
    train_res = {k: v.item() for k,v in train_res.items()}
    train_res['loss'] = train_loss_acc / len(train_loader)

    val_res = evaluate(model, val_loader, val_metrics)

    # Log
    cur_lr = optimizer.param_groups[0]['lr']
    history.append({'epoch': epoch, 'split': 'train', 'lr': cur_lr} | train_res)
    history.append({'epoch': epoch, 'split': 'val', 'lr': cur_lr} | val_res)

    # Scheduler & Early Stopping
    scheduler.step(val_res['loss'])
    early_stopping(val_res['loss'])

    pbar.set_postfix({'t_loss': f"{train_res['loss']:.2f}",
                      'v_loss': f"{val_res['loss']:.2f}",
                      'v_r2': f"{val_res['r2']:.2f}"})

    if early_stopping.early_stop:
        print("Early stopping triggered.")
        break

# DataFrame History
df_hist = pd.DataFrame(history)
df_hist

Training dinamics

In [ ]:
metrics_to_plot = {
    'loss': 'MSE Loss',
    'r2': 'R2 Score',
    'spear': 'Spearman'
}

for y_col, title in metrics_to_plot.items():
    plt.figure(figsize=(10, 6))
    sns.lineplot(data=df_hist, x='epoch', y=y_col, hue='split')
    plt.title(title)

    # Specific styling for R2
    if y_col == 'r2':
        plt.ylim(0, 1)

    plt.show()

In [ ]:
model.eval()
test_res = evaluate(model, test_loader, test_metrics)
print("\nFinal Test Set Results:")
for k, v in test_res.items():
    print(f"{k:>10}: {v:.4f}")

# Compare distributions
preds = []
actuals = []
with torch.no_grad():
    for x, y in test_loader:
        preds.extend(model(x.to(device)).cpu().numpy().flatten())
        actuals.extend(y.numpy().flatten())

plt.figure(figsize=(6,6))
plt.scatter(actuals, preds, alpha=0.5)
plt.plot([min(actuals), max(actuals)], [min(actuals), max(actuals)], 'r--')
plt.xlabel("Actual LogP")
plt.ylabel("Predicted LogP")
plt.title(f"Test Set Parity Plot ({SPLIT})")
plt.show()

# Exhaustive Evaluation (Benchmarking)

In [ ]:
# Define the splits to benchmark
target_splits = ['split_random', 'split_stratified', 'split_adversarial']
benchmark_results = []

print("\n" + "="*40)
print("STARTING BENCHMARK LOOP")
print("="*40)

for split_name in target_splits:
    print(f"\n>>> Training on split: {split_name}")

    # 1. Get Loaders for this specific split
    t_loader, v_loader, te_loader = get_dataloaders(split_name, x_all, y_all, df)

    # 2. Re-initialize Model (Fresh Weights)
    model_bench = MLP(input_dim, hidden_size=128, num_layers=3, dropout=0.2).to(device)

    # 3. Re-initialize Optimization tools
    optimizer_bench = torch.optim.AdamW(model_bench.parameters(), lr=0.001, weight_decay=1e-2)
    scheduler_bench = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_bench, mode='min', factor=0.5, patience=5)
    early_stopping_bench = EarlyStopping(patience=15, verbose=False)

    # Metrics (reset for this run)
    bench_train_metrics = metrics_template.clone().to(device)
    bench_val_metrics = metrics_template.clone().to(device)
    bench_test_metrics = metrics_template.clone().to(device)

    # 4. Training Loop
    # We use a silent loop (no progress bar) to keep output clean,
    # or a simple tqdm if desired.
    for epoch in tqdm.tqdm(range(100), desc=f"Ep {split_name}", leave=False):
        model_bench.train()
        bench_train_metrics.reset()

        for x, y in t_loader:
            x, y = x.to(device), y.to(device)
            optimizer_bench.zero_grad()
            preds = model_bench(x)
            loss = loss_fn(preds, y)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model_bench.parameters(), 1.0)
            optimizer_bench.step()

        # Validation
        val_res_bench = evaluate(model_bench, v_loader, bench_val_metrics)

        # Scheduler & ES
        scheduler_bench.step(val_res_bench['loss'])
        early_stopping_bench(val_res_bench['loss'])

        if early_stopping_bench.early_stop:
            break

    # 5. Final Evaluation on Test Set
    test_res_bench = evaluate(model_bench, te_loader, bench_test_metrics)

    # Store results
    result_entry = {'Split Type': split_name}
    result_entry.update(test_res_bench)
    benchmark_results.append(result_entry)

    print(f"Finished {split_name}. Test R2: {test_res_bench['r2']:.4f}")

# --- Generate Table ---
df_benchmark = pd.DataFrame(benchmark_results)
df_benchmark

In [ ]:
# Reorder columns for readability
cols = ['Split Type', 'loss', 'mae', 'r2', 'spear']
df_benchmark = df_benchmark[cols]

print("\n" + "="*40)
print("FINAL BENCHMARK RESULTS")
print("="*40)
ipy_display.display(df_benchmark)

# Optional: Visualize the comparison
plt.figure(figsize=(8,5))
sns.barplot(data=df_benchmark, x='Split Type', y='r2', hue='Split Type', palette='Set2')
plt.title("R2 Score across Data Splits")
plt.ylim(0, 1)
plt.show()
